In [15]:
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("data/heart.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [3]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [9]:
df_train = df.sample(frac=0.8, random_state=42)  
df_test = df.drop(df_train.index)
df_validation = df_train.sample(frac=0.2, random_state=42)
df_train = df_train.drop(df_validation.index)

In [13]:
target = 'HeartDisease'
column_names = ['Age',
                'ChestPainType',
                'Cholesterol',
                'ExerciseAngina',
                'FastingBS',
                'MaxHR',
                'Oldpeak',
                'RestingBP',
                'RestingECG',
                'ST_Slope',
                'Sex']
cat_features = ['ChestPainType',
                'ExerciseAngina',
                'FastingBS',
                'RestingECG',
                'ST_Slope',
                'Sex']

train_pool = Pool(df_train[column_names], label=df_train[target], cat_features=cat_features,
                  feature_names=column_names)
val_pool = Pool(df_validation[column_names], label=df_validation[target], cat_features=cat_features,
                feature_names=column_names)
model = CatBoostClassifier(iterations=100, depth=3, learning_rate=0.1, verbose=True)
model.fit(train_pool)
model.fit(train_pool, eval_set=val_pool, use_best_model=True, early_stopping_rounds=50)

0:	learn: 0.6655912	total: 56.7ms	remaining: 5.61s
1:	learn: 0.6392368	total: 57.1ms	remaining: 2.8s
2:	learn: 0.6070493	total: 58ms	remaining: 1.88s
3:	learn: 0.5776009	total: 58.9ms	remaining: 1.41s
4:	learn: 0.5558613	total: 59.2ms	remaining: 1.12s
5:	learn: 0.5368728	total: 60ms	remaining: 941ms
6:	learn: 0.5196707	total: 60.9ms	remaining: 809ms
7:	learn: 0.5019170	total: 61.1ms	remaining: 703ms
8:	learn: 0.4866061	total: 62ms	remaining: 627ms
9:	learn: 0.4704827	total: 62.9ms	remaining: 566ms
10:	learn: 0.4574223	total: 63.1ms	remaining: 511ms
11:	learn: 0.4443962	total: 64.1ms	remaining: 470ms
12:	learn: 0.4331834	total: 64.2ms	remaining: 430ms
13:	learn: 0.4237453	total: 64.3ms	remaining: 395ms
14:	learn: 0.4135003	total: 64.5ms	remaining: 365ms
15:	learn: 0.4061741	total: 64.6ms	remaining: 339ms
16:	learn: 0.3994059	total: 64.8ms	remaining: 316ms
17:	learn: 0.3941993	total: 65ms	remaining: 296ms
18:	learn: 0.3887913	total: 65.2ms	remaining: 278ms
19:	learn: 0.3846068	total: 66m

In [17]:
predictions = model.predict(df_test[column_names])
print(classification_report(df_test[target], predictions))

              precision    recall  f1-score   support

           0       0.93      0.75      0.83        92
           1       0.79      0.95      0.86        92

    accuracy                           0.85       184
   macro avg       0.86      0.85      0.85       184
weighted avg       0.86      0.85      0.85       184

